# BC Data Collection

In this notebook, we'll programmatically get data from Business Central through the BC ODataV4 endpoints. These endpoints are exposed in BC Web Services and allows access to different datasets.

## The BC API Utility

In general, getting data from BC is a two-step process. You first need to get a Bearer token that will be used for authentication. Only select accounts can get this and you will need to enter a password as well. Next, you will have to pass this token to another request. This time you need to select the URL (or API endpoint) which will handle your specific data requirement and provide the token as part of the request headers.

To do this, we built the BC API utility to make the process a bit more streamlined, especially for integration in our Airflow scripts.

This contains two classes:

1. `BCTokenClient` which helps us get authentication tokens for further access to BC data. 
2. `BCHandler` which allows us to specify the data we're requesting. We'll pass the token when we first build the class so it would authenticate all of our subsequent requests. The token expires after sometime, to which you'll need to create another one using the `BCTokenClient`.


In [ ]:
import os
from bc_api.authenticator import BCTokenClient 
from bc_api.handler import BCHandler
from dotenv import load_dotenv

import pandas as pd
pd.set_option('display.max_columns', None)

We stored the variables in the `.env` file. This way, we avoid hard coding any secrets inside the notebook. Another good thing is that if we use git, we can set this file inside the `.gitignore` config file, protecting us from any scenario where we might accidentally share secret information, such as passwords and keys.

In [2]:
# Load credentials from .env file
def load_credentials():
    """Load credentials from environment variables."""
    username = os.getenv('BC_USERNAME') or os.getenv('USER')
    password = os.getenv('BC_PASSWORD') or os.getenv('PASSWORD')
    
    if not username:
        raise ValueError("Username not found. Please set BC_USERNAME or USER environment variable.")
    
    if not password:
        raise ValueError("Password not found. Please set BC_PASSWORD or PASSWORD environment variable.")
    
    return username, password


Let's request data using the utilities.

In [3]:
username, password = load_credentials()

tenant_id = '40773bd8-d9fe-4745-9d44-851f81912eb4'
env_name = 'DaconProduction'
company_name = 'Dacon Corporation'

token = BCTokenClient().get_token(username=username, password=password)

bc = BCHandler(tenant_id=tenant_id, env_name=env_name, company_name=company_name, token=token)

c:\Users\laguidote\Documents\2025 Projects\bc-conn\.venv\lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.70.231'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [4]:
data = bc.get_data('VendorLedgerEntries')

c:\Users\laguidote\Documents\2025 Projects\bc-conn\.venv\lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.businesscentral.dynamics.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Now, let's parse the results from the BC dataframe.

In [6]:
pd.json_normalize(data['value'])

Entry_No  Transaction_No Vendor_No Posting_Date    Due_Date  \
0            6               3   SH00001   2025-01-20  2025-01-20   
1           18               4    V00057   2025-01-06  2025-01-06   
2           22               6    V00378   2025-01-09  2025-01-09   
3           26               8    V00360   2025-01-15  2025-01-15   
4           34              12    V00057   2025-01-16  2025-01-16   
...        ...             ...       ...          ...         ...   
2493     11753            3440    V00106   2025-10-15  2025-10-15   
2494     11759            3443    V00054   2025-10-09  2025-10-09   
2495     11763            3444    V00010   2025-10-17  2025-10-17   
2496     11764            3444    V00010   2025-10-17  2025-10-17   
2497     11765            3444    V00010   2025-10-17  2025-10-17   

     Pmt_Discount_Date Document_Date Document_Type   Document_No  \
0           0001-01-01    2025-01-20                  AR00002671   
1           2025-01-06    2025-01-06       Invoice  PPIN00000001   
2           0001-01-01    2025-01-09       Payment       1862364   
3           0001-01-01    2025-01-15       Payment       1862368   
4           0001-01-01    2025-01-16       Payment       1862373   
...                ...           ...           ...           ...   
2493        0001-01-01    2025-10-15                PPIN00000614   
2494        0001-01-01    2025-10-09       Payment      DM000172   
2495        0001-01-01    2025-10-17                  JV00000711   
2496        0001-01-01    2025-10-17                  JV00000711   
2497        0001-01-01    2025-10-17                  JV00000711   

     Purchaser_Code Source_Code Reason_Code IC_Partner_Code   Open  \
0                    CASHRECJNL                               True   
1                     PURCHASES                              False   
2                    PAYMENTJNL                              False   
3                    PAYMENTJNL                              False   
4                    PAYMENTJNL                              False   
...             ...         ...         ...             ...    ...   
2493                     GENJNL                              False   
2494                 PAYMENTJNL                               True   
2495                 PAYMENTJNL                               True   
2496                 PAYMENTJNL                               True   
2497                 PAYMENTJNL                               True   

     Currency_Code  Dimension_Set_ID        Amount  Debit_Amount  \
0                                 30 -8.333330e+03          0.00   
1                                 42 -2.629634e+04          0.00   
2                                 80  1.008000e+05     100800.00   
3                                105  5.206345e+04      52063.45   
4                                168  2.629634e+04      26296.34   
...            ...               ...           ...           ...   
2493                            1976 -1.500000e+05          0.00   
2494                            4366  5.027846e+05     502784.57   
2495                            5938 -1.022222e+06          0.00   
2496                            5505 -1.479278e+06          0.00   
2497                            5505 -2.000000e+08          0.00   

      Credit_Amount  Remaining_Amount    Amount_LCY  Debit_Amount_LCY  \
0      8.333330e+03     -8.333330e+03 -8.333330e+03              0.00   
1      2.629634e+04      0.000000e+00 -2.629634e+04              0.00   
2      0.000000e+00      0.000000e+00  1.008000e+05         100800.00   
3      0.000000e+00      0.000000e+00  5.206345e+04          52063.45   
4      0.000000e+00      0.000000e+00  2.629634e+04          26296.34   
...             ...               ...           ...               ...   
2493   1.500000e+05      0.000000e+00 -1.500000e+05              0.00   
2494   0.000000e+00      5.027846e+05  5.027846e+05         502784.57   
2495   1.022222e+06     -1.022222e+06 -1.0222

## Metadata Collection

In [ ]:
import requests
import xml.etree.ElementTree as ET

In [ ]:
tenant_id = '40773bd8-d9fe-4745-9d44-851f81912eb4'
env_name = 'DaconProduction'
token = BCTokenClient().get_token(username=username, password=password)

endpoint = f'https://api.businesscentral.dynamics.com/v2.0/{tenant_id}/{env_name}/ODataV4/$metadata'
headers = {'Authorization': f'Bearer {token}'}

data = requests.get(endpoint, headers=headers).content.decode('utf-8')
data

c:\Users\laguidote\Documents\2025 Projects\bc-conn\.venv\lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.70.231'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


'<?xml version="1.0" encoding="utf-8"?><edmx:Edmx Version="4.0" xmlns:edmx="http://docs.oasis-open.org/odata/ns/edmx"><edmx:DataServices><Schema Namespace="NAV" xmlns="http://docs.oasis-open.org/odata/ns/edm"><EntityType Name="Media"><Key><PropertyRef Name="ID" /></Key><Property Name="ID" Type="Edm.Guid" Nullable="false"><Annotation Term="NAV.LabelId" String="ID" /><Annotation Term="NAV.NavType"><EnumMember>NAV.NavType/Guid</EnumMember></Annotation><Annotation Term="NAV.AllowEditOnCreate" Bool="true" /></Property><Property Name="timestamp" Type="Edm.Int64"><Annotation Term="NAV.LabelId" String="timestamp" /><Annotation Term="NAV.NavType"><EnumMember>NAV.NavType/Long</EnumMember></Annotation><Annotation Term="NAV.AllowEdit" Bool="false" /><Annotation Term="NAV.AllowEditOnCreate" Bool="false" /></Property><Property Name="Description" Type="Edm.String" MaxLength="250"><Annotation Term="NAV.LabelId" String="Description" /><Annotation Term="NAV.NavType"><EnumMember>NAV.NavType/String</EnumM

In [ ]:
ns = {
    'edmx': 'http://docs.oasis-open.org/odata/ns/edmx',
    'edm': 'http://docs.oasis-open.org/odata/ns/edm'
}

root = ET.fromstring(data)

entities = {}

for entity in root.findall('.//edm:EntityType', ns):
    entity_name = entity.attrib.get('Name')
    entities[entity_name] = {}

    for prop in entity.findall('edm:Property', ns):
        prop_name = prop.attrib.get('Name')
        prop_type = prop.attrib.get('Type')
        prop_null = prop.attrib.get('Nullable')
        prop_maxlen = prop.attrib.get('MaxLength')
        prop_scale = prop.attrib.get('Scale')

        annotations = {}
        for ann in prop.findall('edm:Annotation', ns):
            term = ann.attrib.get('Term')
            value = ann.attrib.get('String') or ann.attrib.get('Bool')
            enum = ann.find('edm:EnumMember', ns)
            if enum is not None:
                value = enum.text
            annotations[term] = value

        entities[entity_name][prop_name] = {
            'Type': prop_type,
            'Nullable': prop_null,
            'MaxLength': prop_maxlen,
            'Scale': prop_scale,
            'Annotations': annotations
        }

In [ ]:
rows = []
for prop, info in entities.items():
    for field, details in info.items():
        row = {'API Endpoint Name': prop}
        row['Column Name'] = field
        row['DataType'] = details.get('Type', None)
        row['Nullable'] = details.get('Nullable', None)
        row['MaxLength'] = details.get('MaxLength', None)
        row['Scale'] = details.get('Scale', None)
        for k, v in details.get('Annotations', {}).items():
            row[f'Annotations.{k}'] = v
        rows.append(row)

In [ ]:
df_metadata = pd.json_normalize(rows)
df_metadata.to_csv('bc_metadata.csv', index=False)